In [1]:

import pandas as pd
import numpy as np

# load Billionaires mapping list

This is a list of id mappings between Forbes and Hurun

In [2]:
# all_ids_v1 = pd.read_csv('../source/billionaires list - eo_final.csv', na_values=['', 'n/a'])
all_ids = pd.read_csv('../source/GM-Billionaires_list-v2 - eo_final.csv', na_values=['', 'n/a'])

In [3]:
all_ids = all_ids.iloc[:, :3]

In [4]:
# all_ids_v1[all_ids_v1['Unique Gapminder ID'].duplicated(keep=False)]

In [5]:
# all_ids_v1.shape

In [6]:
all_ids.shape

(6053, 3)

In [7]:
all_ids.drop_duplicates()

,Unique Gapminder ID,Forbes_id,Hurun_id
0,abdulla_al_futtaim,abdulla_al_futtaim,abdulla_al_futtaim
1,abdullah_al_rajhi,abdullah_al_rajhi,abdullah_al_rajhi
2,abdulsamad_rabiu,abdulsamad_rabiu,abdulsamad_rabiu
3,abhay_soi,abhay_soi,abhay_soi
4,abhay_vakil,abhay_vakil,abhay_vakil
...,...,...,...
6048,zhu_chongyun,zhu_chongyun,NaN
6049,zhu_qingyi,zhu_qingyi,NaN
6050,zhu_yan,zhu_yan,NaN
6051,ziyad_manasir,ziyad_manasir,NaN


In [8]:
all_ids = all_ids.dropna(axis=0, how='all').drop_duplicates()

In [9]:
all_ids.columns = ['gm_id', 'forbes_id', 'hurun_id']

# Create Datapoints


1. convert indicators to use gm id
2. merge indicators from forbes and hurun


for some of forbes records, we need to split them into multiple records (i.e. forbes has entity as a group of people, but hurun have separated record for each person in list)
so we should first divide forbes mapping into 2 parts.


In [10]:
# how to calculate the indicator required by income mountain:
# income by person, year -> income by person, geo, year -> income_level by person, geo, year
# -> population by income_level, geo, year

In [11]:
fm = all_ids[['forbes_id', 'gm_id']].dropna(how='any')

fm1 = fm[fm['forbes_id'].duplicated(keep=False)]  # these ones need to be split to mulitple people.
fm2 = fm[~fm['forbes_id'].duplicated(keep=False)]

In [12]:
fm1

,forbes_id,gm_id
2950,chuchat_petaumpai_daonapa_petampai,daonapa_petampai
3112,lee_yeow_chor_yeow_seng,lee_yeow_chor
3230,safra,joseph_safra
3277,udo_harald_tschira,udo_tschira
3278,vinod_anil_rai_gupta,vinod_gupta
3301,willi_isolde_liebherr,willi_liebherr
3424,vinod_anil_rai_gupta,anil_rai_gupta
3532,safra,chelia_cohen_safra
3602,chuchat_petaumpai_daonapa_petampai,chuchat_petaumpai
3625,safra,david_j_safra


In [13]:
fm1.sort_values(by='gm_id')

,forbes_id,gm_id
3424,vinod_anil_rai_gupta,anil_rai_gupta
3532,safra,chelia_cohen_safra
3602,chuchat_petaumpai_daonapa_petampai,chuchat_petaumpai
2950,chuchat_petaumpai_daonapa_petampai,daonapa_petampai
3625,safra,david_j_safra
3698,safra,esther_safra
3816,udo_harald_tschira,harald_tschira
3917,willi_isolde_liebherr,isolde_liebherr
3924,safra,jacob_j_safra
3230,safra,joseph_safra


In [14]:
# 1. convert forbes datapoints to use gm_id
# 1.1 split groups

In [15]:
forbes_dps = pd.read_csv('../intermediate/forbes/ddf--datapoints--annual_income--by--person--year.csv')

In [16]:
forbes_dps

,person,year,annual_income
0,a_jayson_adair,2021,30000000.0
1,a_jayson_adair,2022,33000000.0
2,a_jerrold_perenchio,2002,78000000.0
3,a_jerrold_perenchio,2003,69000000.0
4,a_jerrold_perenchio,2004,81000000.0
...,...,...,...
28720,zygmunt_solorz_zak,2018,84000000.0
28721,zygmunt_solorz_zak,2019,81000000.0
28722,zygmunt_solorz_zak,2020,72000000.0
28723,zygmunt_solorz_zak,2021,96000000.0


In [17]:
fb1 = forbes_dps[forbes_dps.person.isin(fm1.forbes_id.unique())].copy()
fb2 = forbes_dps[~forbes_dps.person.isin(fm1.forbes_id.unique())].copy()

In [18]:
forbes_dps.shape

(28725, 3)

In [19]:
fb1.shape[0] + fb2.shape[0]

28725

In [20]:
fm1g = fm1.groupby('forbes_id')['gm_id']

In [21]:
fm1g.get_group('kwee')

4076     kwee_liong_keng
4077    kwee_liong_phing
4078     kwee_liong_seen
4079      kwee_liong_tek
5618                kwee
Name: gm_id, dtype: object

In [22]:
fb1g = fb1.groupby('person')

In [23]:
fb1g.get_group('lee_yeow_chor_yeow_seng')

,person,year,annual_income
14795,lee_yeow_chor_yeow_seng,2020,108000000.0
14796,lee_yeow_chor_yeow_seng,2021,153000000.0


In [24]:
def create_rec(df, names):
    """split the record by names evenly. create dataframe which row number equal to number of names."""
    recs = list()
    ser = df.set_index('year')['annual_income']
    num = len(names)
    
    for y in ser.index:
        income = int(ser.loc[y] / num)
        for n in names:
            rec = dict(person=n, year=y, annual_income=income)
            recs.append(rec)
    return pd.DataFrame.from_records(recs)

In [25]:
fm1g.get_group('lee_yeow_chor_yeow_seng').values

array(['lee_yeow_chor', 'lee_yeow_seng'], dtype=object)

In [26]:
fb3dfs = list()

for g, df in fb1g:
    new_df = create_rec(df, fm1g.get_group(g).values)
    fb3dfs.append(new_df)

In [27]:
fb3 = pd.concat(fb3dfs, ignore_index=True)

In [28]:
fb3

,person,year,annual_income
0,daonapa_petampai,2018,30000000
1,chuchat_petaumpai,2018,30000000
2,daonapa_petampai,2019,36000000
3,chuchat_petaumpai,2019,36000000
4,daonapa_petampai,2021,52500000
...,...,...,...
109,isolde_liebherr,2016,75000000
110,willi_liebherr,2017,72000000
111,isolde_liebherr,2017,72000000
112,willi_liebherr,2018,106500000


In [29]:
fb3[fb3['person'].isin(['lee_yeow_chor', 'lee_yeow_seng'])]

,person,year,annual_income
58,lee_yeow_chor,2020,54000000
59,lee_yeow_seng,2020,54000000
60,lee_yeow_chor,2021,76500000
61,lee_yeow_seng,2021,76500000


In [30]:
# 1.2 convert remaining records to use gm id

In [31]:
np.any(fm2.set_index('forbes_id')['gm_id'].index.duplicated())  # Should be false.

False

In [32]:
fb2_map = fm2.set_index('forbes_id')['gm_id'].to_dict()

In [33]:
# NOTE: if below failed, then there are not existing entries, we should exam what happened

fb2['gm_id'] = fb2['person'].map(lambda x: fb2_map[x])

In [34]:
forbes_dps[forbes_dps['person'] == 'xiangdong_qi']  # should be non empty dataframe

,person,year,annual_income
27503,xiangdong_qi,2022,63000000.0


In [35]:
def _f(x):
    if x not in fb2_map:
        print(x)
        return None
    return fb2_map[x]
_ = fb2['person'].map(_f)

In [36]:
fb2

,person,year,annual_income,gm_id
0,a_jayson_adair,2021,30000000.0,a_jayson_adair
1,a_jayson_adair,2022,33000000.0,a_jayson_adair
2,a_jerrold_perenchio,2002,78000000.0,a_jerrold_perenchio
3,a_jerrold_perenchio,2003,69000000.0,a_jerrold_perenchio
4,a_jerrold_perenchio,2004,81000000.0,a_jerrold_perenchio
...,...,...,...,...
28720,zygmunt_solorz_zak,2018,84000000.0,zygmunt_solorz_zak
28721,zygmunt_solorz_zak,2019,81000000.0,zygmunt_solorz_zak
28722,zygmunt_solorz_zak,2020,72000000.0,zygmunt_solorz_zak
28723,zygmunt_solorz_zak,2021,96000000.0,zygmunt_solorz_zak


In [37]:
fb2[pd.isnull(fb2.gm_id)]['person'].unique()

array([], dtype=object)

In [38]:
fb4 = fb2[['gm_id', 'year', 'annual_income']].copy()
# uncomment to drop all unknown entities
# fb4 = fb4.dropna(how='any')
fb4.columns = ['person', 'year', 'annual_income']

In [39]:
fb_final = pd.concat([fb4, fb3], ignore_index=True).sort_values(by=['person', 'year'])

In [40]:
fb_final

,person,year,annual_income
0,a_jayson_adair,2021,30000000.0
1,a_jayson_adair,2022,33000000.0
2,a_jerrold_perenchio,2002,78000000.0
3,a_jerrold_perenchio,2003,69000000.0
4,a_jerrold_perenchio,2004,81000000.0
...,...,...,...
28681,zygmunt_solorz_zak,2018,84000000.0
28682,zygmunt_solorz_zak,2019,81000000.0
28683,zygmunt_solorz_zak,2020,72000000.0
28684,zygmunt_solorz_zak,2021,96000000.0


In [41]:
fb_final[fb_final['person'] == 'adam_neumann']

,person,year,annual_income
153,adam_neumann,2016,45000000.0
154,adam_neumann,2017,75000000.0
155,adam_neumann,2018,75000000.0
156,adam_neumann,2019,123000000.0
157,adam_neumann,2022,42000000.0


In [42]:
# done for forbes datapoints

In [43]:
# now convert hurun data to use gm_id

In [44]:
hurun_dps = pd.read_csv('../intermediate/hurun/ddf--datapoints--annual_income--by--person--year.csv')

In [45]:
hurun_dps

,person,year,annual_income
0,abdulla_ahmed_al_ghurair,2017,39000000.0
1,abdulla_ahmed_al_ghurair,2018,39000000.0
2,abdulla_ahmed_al_ghurair,2019,36000000.0
3,abdulla_ahmed_al_ghurair,2020,36000000.0
4,abdulla_ahmed_al_ghurair,2021,36000000.0
...,...,...,...
17977,zygmunt_solorz_zak,2018,90000000.0
17978,zygmunt_solorz_zak,2019,78000000.0
17979,zygmunt_solorz_zak,2020,84000000.0
17980,zygmunt_solorz_zak,2021,96000000.0


In [46]:
hr = hurun_dps.copy()

In [47]:
hr_map = all_ids[['hurun_id', 'gm_id']].dropna(how='any').set_index('hurun_id')['gm_id'].to_dict()

In [48]:
# if this failed, we should check which ones are missing.

hr['gm_id'] = hr['person'].map(lambda x: hr_map[x])

In [49]:
hr[hr['person'] == 'zhuo_jun']

,person,year,annual_income,gm_id
17920,zhuo_jun,2017,30000000.0,zhuo_jun_1
17921,zhuo_jun,2018,39000000.0,zhuo_jun_1
17922,zhuo_jun,2019,33000000.0,zhuo_jun_1
17923,zhuo_jun,2020,51000000.0,zhuo_jun_1
17924,zhuo_jun,2021,45000000.0,zhuo_jun_1
17925,zhuo_jun,2022,51000000.0,zhuo_jun_1


In [50]:
def _f(x):
    if x not in hr_map:
        print(x)
        return None
    return hr_map[x]
_ = hr['person'].map(_f)

In [51]:
len(hr[pd.isnull(hr.gm_id)]['person'].unique())  # dropped records.

0

In [52]:
len(hr.person.unique())  # total records

5028

In [53]:
hr2 = hr[['gm_id', 'year', 'annual_income']].dropna()

In [54]:
hr2

,gm_id,year,annual_income
0,abdulla_bin_ahmad_al_ghurair,2017,39000000.0
1,abdulla_bin_ahmad_al_ghurair,2018,39000000.0
2,abdulla_bin_ahmad_al_ghurair,2019,36000000.0
3,abdulla_bin_ahmad_al_ghurair,2020,36000000.0
4,abdulla_bin_ahmad_al_ghurair,2021,36000000.0
...,...,...,...
17977,zygmunt_solorz_zak,2018,90000000.0
17978,zygmunt_solorz_zak,2019,78000000.0
17979,zygmunt_solorz_zak,2020,84000000.0
17980,zygmunt_solorz_zak,2021,96000000.0


In [55]:
hr2.columns = ['person', 'year', 'annual_income']

In [56]:
hr2[hr2['person'] == 'adam_neuman']

,person,year,annual_income
66,adam_neuman,2017,78000000.0
67,adam_neuman,2018,96000000.0


In [57]:
# in hurun list, there are many entities without an country property. let's drop them

In [58]:
hurun_ent = pd.read_csv('../intermediate/hurun/ddf--entities--person.csv')

In [59]:
hurun_ent.columns

Index(['person', 'name', 'birth', 'birth_1', 'birth_2', 'birth_3', 'age',
       'sex', 'permanent_country', 'industry', 'company', 'country', 'geo',
       'birth_comb'],
      dtype='object')

In [60]:
hascountry = ~pd.isnull(hurun_ent['geo'])

ppl_hascountry = hurun_ent[hascountry]['person'].values

In [61]:
hr3 = hr2[hr2['person'].isin(ppl_hascountry)]

In [62]:
hr3

,person,year,annual_income
6,abdulla_al_futtaim,2017,93000000.0
7,abdulla_al_futtaim,2018,93000000.0
8,abdulla_al_futtaim,2019,93000000.0
9,abdulla_al_futtaim,2020,36000000.0
10,abdulla_al_futtaim,2021,45000000.0
...,...,...,...
17977,zygmunt_solorz_zak,2018,90000000.0
17978,zygmunt_solorz_zak,2019,78000000.0
17979,zygmunt_solorz_zak,2020,84000000.0
17980,zygmunt_solorz_zak,2021,96000000.0


In [63]:
# done with hurun data

In [64]:
# now, concat all values, drop duplicated

In [65]:
d1 = fb_final.copy()
d1['source'] = 'forbes'

d2 = hr3.copy()
d2['source'] = 'hurun'

dps = pd.concat([d1, d2], ignore_index=True)

In [66]:
dps[dps.duplicated(subset=['person', 'year'], keep=False)]

,person,year,annual_income,source
43,abdulla_al_futtaim,2017,123000000.0,forbes
44,abdulla_al_futtaim,2018,99000000.0,forbes
45,abdulla_al_futtaim,2019,75000000.0,forbes
46,abdulla_al_futtaim,2020,63000000.0,forbes
47,abdulla_al_futtaim,2021,66000000.0,forbes
...,...,...,...,...
43029,zygmunt_solorz_zak,2018,90000000.0,hurun
43030,zygmunt_solorz_zak,2019,78000000.0,hurun
43031,zygmunt_solorz_zak,2020,84000000.0,hurun
43032,zygmunt_solorz_zak,2021,96000000.0,hurun


In [67]:
# let's see how different are they
dpsg = dps.groupby(['person'])

In [68]:
dpsg.get_group('mark_zuckerberg').sort_values(by=['year', 'source'])   ## TODO: maybe check more people

,person,year,annual_income,source
16995,mark_zuckerberg,2008,4.500000e+07,forbes
16996,mark_zuckerberg,2010,1.200000e+08,forbes
16997,mark_zuckerberg,2011,4.050000e+08,forbes
16998,mark_zuckerberg,2012,5.250000e+08,forbes
36886,mark_zuckerberg,2012,7.800000e+08,hurun
16999,mark_zuckerberg,2013,3.990000e+08,forbes
36887,mark_zuckerberg,2013,5.100000e+08,hurun
17000,mark_zuckerberg,2014,8.550000e+08,forbes
36888,mark_zuckerberg,2014,9.300000e+08,hurun
17001,mark_zuckerberg,2015,1.002000e+09,forbes


In [69]:
# But I am going to use Forbes data for these duplicated people for now.

In [70]:
def keep_one_source(df):
    if len(df['source'].unique()) == 2:
        df_new = df[df['source'] == 'forbes'][['person', 'year', 'annual_income']].set_index(['person', 'year'])
    else:
        df_new = df[['person', 'year', 'annual_income']].set_index(['person', 'year'])
    return df_new

In [71]:
keep_one_source(dpsg.get_group('mark_zuckerberg'))

annual_income
person          year               
mark_zuckerberg 2008   4.500000e+07
                2010   1.200000e+08
                2011   4.050000e+08
                2012   5.250000e+08
                2013   3.990000e+08
                2014   8.550000e+08
                2015   1.002000e+09
                2016   1.338000e+09
                2017   1.680000e+09
                2018   2.130000e+09
                2019   1.869000e+09
                2020   1.641000e+09
                2021   2.910000e+09
                2022   2.019000e+09

In [72]:
dps2 = dpsg.apply(keep_one_source)

In [73]:
dps2

annual_income
person              person              year               
a_jayson_adair      a_jayson_adair      2021     30000000.0
                                        2022     33000000.0
a_jerrold_perenchio a_jerrold_perenchio 2002     78000000.0
                                        2003     69000000.0
                                        2004     81000000.0
...                                                     ...
zygmunt_solorz_zak  zygmunt_solorz_zak  2018     84000000.0
                                        2019     81000000.0
                                        2020     72000000.0
                                        2021     96000000.0
                                        2022     87000000.0

[32436 rows x 1 columns]

In [74]:
dps2 = dps2.reset_index(0, drop=True)

In [75]:
dps2

annual_income
person              year               
a_jayson_adair      2021     30000000.0
                    2022     33000000.0
a_jerrold_perenchio 2002     78000000.0
                    2003     69000000.0
                    2004     81000000.0
...                                 ...
zygmunt_solorz_zak  2018     84000000.0
                    2019     81000000.0
                    2020     72000000.0
                    2021     96000000.0
                    2022     87000000.0

[32436 rows x 1 columns]

In [76]:
dps2['annual_income'] = dps2['annual_income'].astype(int)

In [77]:
dps2 = dps2.reset_index()

In [78]:
dps2.columns = ['person', 'time', 'annual_income']

In [146]:
# now we should convert them to 2017 PPP dollars.
rates = [1.652, 1.671, 1.658, 1.620, 1.563, 1.542, 1.513, 1.462, 1.438, 1.437, 1.449, 1.427, 1.376, 1.347, 1.337, 1.279, 1.202, 1.162, 1.142, 1.137, 1.111, 1.102, 1.091, 1.068, 1.040, 1.028, 1.012, 1.019, 1.019, 1.000, 0.955, 0.935, 0.917, 0.88, 0.86]
years = list(range(1988, 2023))
rs = pd.Series(rates, index=years)

In [147]:
rs

1988    1.652
1989    1.671
1990    1.658
1991    1.620
1992    1.563
1993    1.542
1994    1.513
1995    1.462
1996    1.438
1997    1.437
1998    1.449
1999    1.427
2000    1.376
2001    1.347
2002    1.337
2003    1.279
2004    1.202
2005    1.162
2006    1.142
2007    1.137
2008    1.111
2009    1.102
2010    1.091
2011    1.068
2012    1.040
2013    1.028
2014    1.012
2015    1.019
2016    1.019
2017    1.000
2018    0.955
2019    0.935
2020    0.917
2021    0.880
2022    0.860
dtype: float64

In [148]:
rs.index.name = 'time'
rs.name = 'rate'
dfrs = rs.reset_index()

In [149]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [150]:
X = rs.index.values
Y = rs.values
X = sm.add_constant(X)

model = sm.OLS(Y, X)
modelres = model.fit()

In [153]:
b, a = modelres.params

def get_inflation_rate(y):
    if y in rs.index:
        return rs.loc[y]
    else:
        return a * y + b

In [154]:
dps2['rates'] = dps2['time'].map(get_inflation_rate)

In [155]:
dps2['annual_income_2017'] = dps2['annual_income'] * dps2['rates']

In [156]:
dps2

,person,time,annual_income,rates,annual_income_2017
0,a_jayson_adair,2021,30000000,0.880,26400000.0
1,a_jayson_adair,2022,33000000,0.860,28380000.0
2,a_jerrold_perenchio,2002,78000000,1.337,104286000.0
3,a_jerrold_perenchio,2003,69000000,1.279,88251000.0
4,a_jerrold_perenchio,2004,81000000,1.202,97362000.0
...,...,...,...,...,...
32431,zygmunt_solorz_zak,2018,84000000,0.955,80220000.0
32432,zygmunt_solorz_zak,2019,81000000,0.935,75735000.0
32433,zygmunt_solorz_zak,2020,72000000,0.917,66024000.0
32434,zygmunt_solorz_zak,2021,96000000,0.880,84480000.0


In [157]:
dps3 = dps2.set_index(['person', 'time'])['annual_income_2017']

In [158]:
dps3.name = 'annual_income'

In [159]:
dps3

person               time
a_jayson_adair       2021     26400000.0
                     2022     28380000.0
a_jerrold_perenchio  2002    104286000.0
                     2003     88251000.0
                     2004     97362000.0
                                ...     
zygmunt_solorz_zak   2018     80220000.0
                     2019     75735000.0
                     2020     66024000.0
                     2021     84480000.0
                     2022     74820000.0
Name: annual_income, Length: 32436, dtype: float64

In [160]:
dps3.astype(int).to_csv('../../ddf--datapoints--annual_income--by--person--time.csv', index=False)

In [80]:
dps2 

,person,time,annual_income
0,a_jayson_adair,2021,30000000
1,a_jayson_adair,2022,33000000
2,a_jerrold_perenchio,2002,78000000
3,a_jerrold_perenchio,2003,69000000
4,a_jerrold_perenchio,2004,81000000
...,...,...,...
32431,zygmunt_solorz_zak,2018,84000000
32432,zygmunt_solorz_zak,2019,81000000
32433,zygmunt_solorz_zak,2020,72000000
32434,zygmunt_solorz_zak,2021,96000000


In [81]:
# now get the daily income 
# FIXME: should start from worth, then annual_income/daily_income

In [161]:
dps4 = dps3.copy()

In [162]:
dps4 = dps4 / 365.
dps4 = dps4.astype(int)

dps4.name = 'daily_income'

In [163]:
dps4.to_csv('../../ddf--datapoints--daily_income--by--person--time.csv', index=False)

In [86]:
# 2. combine the entities

In [87]:
# For forbes list, we should also take care of entity spliting.

In [88]:
forbes_ent = pd.read_csv('../intermediate/forbes/ddf--entities--person.csv')

In [89]:
forbes_ent 

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul
...,...,...,...,...,...,...,...,...,...,...,...
4214,zong_yanmin,Zong Yanmin,China,NaN,NaN,Semiconductor materials,Manufacturing,no-pic,NaN,M,Zong
4215,zugen_ni,Zugen Ni,China,Jiangsu,NaN,appliances,Manufacturing,no-pic,65.0,M,Ni
4216,zuo_hui,Zuo Hui,China,NaN,NaN,real estate services,Real Estate,no-pic,51.0,M,Zuo
4217,zuowen_song,Zuowen Song,China,NaN,NaN,"aluminum, diversified",Diversified,zuowen-song,75.0,M,Song


In [90]:
fbent1 = forbes_ent[forbes_ent.person.isin(fm1.forbes_id.unique())].copy()
fbent2 = forbes_ent[~forbes_ent.person.isin(fm1.forbes_id.unique())].copy()

In [91]:
fbent2

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul
...,...,...,...,...,...,...,...,...,...,...,...
4214,zong_yanmin,Zong Yanmin,China,NaN,NaN,Semiconductor materials,Manufacturing,no-pic,NaN,M,Zong
4215,zugen_ni,Zugen Ni,China,Jiangsu,NaN,appliances,Manufacturing,no-pic,65.0,M,Ni
4216,zuo_hui,Zuo Hui,China,NaN,NaN,real estate services,Real Estate,no-pic,51.0,M,Zuo
4217,zuowen_song,Zuowen Song,China,NaN,NaN,"aluminum, diversified",Diversified,zuowen-song,75.0,M,Song


In [92]:
fbent2.columns

Index(['person', 'name', 'country', 'state', 'headquarters', 'source',
       'industry', 'image_uri', 'age', 'gender', 'last_name'],
      dtype='object')

In [93]:
cols_rec = ['country', 'state', 'headquarters', 'source', 'industry', 'last_name']

In [94]:
def create_entrec():
    res = []
    
    for _, row in fbent1.iterrows():
        name = row['person']
        newnames = fm1g.get_group(name)
        rec = dict([(c, row[c]) for c in cols_rec])
        for n in newnames:
            rec_ = rec.copy()
            rec_['person'] = n
            res.append(rec_)
            
    return pd.DataFrame.from_records(res)

In [95]:
create_entrec()

,country,state,headquarters,source,industry,last_name,person
0,Thailand,NaN,NaN,motorcycle loans,Finance & Investments; Finance and Investments,Chuchat Petaumpai &,daonapa_petampai
1,Thailand,NaN,NaN,motorcycle loans,Finance & Investments; Finance and Investments,Chuchat Petaumpai &,chuchat_petaumpai
2,Singapore,NaN,NaN,Real Estate; Pontiac Land Group; real estate,Real Estate,Kwee,kwee_liong_keng
3,Singapore,NaN,NaN,Real Estate; Pontiac Land Group; real estate,Real Estate,Kwee,kwee_liong_phing
4,Singapore,NaN,NaN,Real Estate; Pontiac Land Group; real estate,Real Estate,Kwee,kwee_liong_seen
5,Singapore,NaN,NaN,Real Estate; Pontiac Land Group; real estate,Real Estate,Kwee,kwee_liong_tek
6,Singapore,NaN,NaN,Real Estate; Pontiac Land Group; real estate,Real Estate,Kwee,kwee
7,Malaysia,NaN,NaN,"palm oil, property",Food & Beverage,Lee,lee_yeow_chor
8,Malaysia,NaN,NaN,"palm oil, property",Food & Beverage,Lee,lee_yeow_seng
9,Brazil,NaN,NaN,banking,Finance & Investments,Safra,joseph_safra


In [96]:
fbent3 = create_entrec()

In [97]:
fbent4 = fbent2.copy()

In [98]:
len(fm2)

4217

In [99]:
fm2_mapping = fm2.set_index('forbes_id')['gm_id'].to_dict()

In [100]:
def _f(x):
    if x in fm2_mapping:
        return fm2_mapping[x]
    print(x)
    return None

fbent4['gm_id'] = fbent4['person'].map(_f)

In [101]:
fbent4[pd.isnull(fbent4.gm_id)]

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,gm_id


In [102]:
fbent4 = fbent4[~pd.isnull(fbent4.gm_id)]
fbent4 = fbent4.set_index(['gm_id']).drop(columns=['person'])
fbent4.index.name = 'person'
fbent4 = fbent4.reset_index()

In [103]:
fbent4

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul
...,...,...,...,...,...,...,...,...,...,...,...
4207,zong_yanmin,Zong Yanmin,China,NaN,NaN,Semiconductor materials,Manufacturing,no-pic,NaN,M,Zong
4208,zugen_ni,Zugen Ni,China,Jiangsu,NaN,appliances,Manufacturing,no-pic,65.0,M,Ni
4209,zuo_hui,Zuo Hui,China,NaN,NaN,real estate services,Real Estate,no-pic,51.0,M,Zuo
4210,zuowen_song,Zuowen Song,China,NaN,NaN,"aluminum, diversified",Diversified,zuowen-song,75.0,M,Song


In [104]:
fbent_final = pd.concat([fbent4, fbent3], ignore_index=True)

In [105]:
fbent_final

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul
...,...,...,...,...,...,...,...,...,...,...,...
4227,harald_tschira,NaN,Germany,NaN,NaN,software; Software,Technology,NaN,NaN,NaN,Tschira
4228,vinod_gupta,NaN,India,NaN,NaN,electrical equipment,Manufacturing,NaN,NaN,NaN,Gupta
4229,anil_rai_gupta,NaN,India,NaN,NaN,electrical equipment,Manufacturing,NaN,NaN,NaN,Gupta
4230,willi_liebherr,NaN,Switzerland,NaN,NaN,Construction; construction,Construction & Engineering,NaN,NaN,NaN,Liebherr


In [106]:
fbent_final[fbent_final.country.str.contains(';')].shape

(52, 11)

In [107]:
fbent_final[fbent_final.country.str.contains(';')]['country'].values

array(['United States; Colombia', 'Israel; Kazakhstan', 'India; Thailand',
       'Germany; United States', 'Czechia; Czech Republic',
       'Lebanon; Saudi Arabia', 'Lebanon; Switzerland', 'Georgia; Russia',
       'Canada; China', 'Hong Kong; China', 'Hong Kong; China',
       'Czechia; Czech Republic', 'Brazil; United States',
       'United States; Russia', 'Switzerland; Germany',
       'Switzerland; Germany', 'Macau; Macao', 'Hong Kong; China',
       'Israel; Russia', 'Switzerland; Australia', 'Singapore; Taiwan',
       'United States; Canada', 'Cyprus; Norway',
       'Czechia; Czech Republic', 'China; Hong Kong', 'Singapore; China',
       'Monaco; Brazil', 'United States; China',
       'Czechia; Czech Republic', 'United States; South Korea',
       'Eswatini (Swaziland); Swaziland', 'China; Hong Kong',
       'Ireland; India', 'Belgium; Kazakhstan', 'Czechia; Czech Republic',
       'Czechia; Czech Republic', 'Czechia; Czech Republic',
       'Czechia; Czech Republic', 'Au

In [108]:
# TODO: double check those reported with in different countries within etl notebook for forbes

In [109]:
hurun_ent = pd.read_csv('../intermediate/hurun/ddf--entities--person.csv')

In [110]:
hurun_map = all_ids[['hurun_id', 'gm_id']].dropna(how='any').set_index('hurun_id')['gm_id'].to_dict()

In [111]:
hurun_ent

,person,name,birth,birth_1,birth_2,birth_3,age,sex,permanent_country,industry,company,country,geo,birth_comb
0,abdulla_ahmed_al_ghurair,Abdulla Ahmed Al Ghurair;Abdulla Ahmed Al Ghur...,NaN,NaN,1955;1956,NaN,NaN,Male,United Arab Emirates-Dubai,Banking,Mashreq Bank,United Arab Emirates,are,1955.0
1,abdulla_al_futtaim,Abdulla Al Futtaim,NaN,NaN,NaN,NaN,NaN,Male,United Arab Emirates-Dubai,Automobile Distribution;Automobiles & Componen...,Al Futtaim,United Arab Emirates,are,NaN
2,abdullah_al_rajhi,Abdullah Al Rajhi,NaN,NaN,NaN,NaN,NaN,Male,Saudi Arabia-Riyadh,Banking,Al Rajhi Banking And Investment,Saudi Arabia,sau,NaN
3,abdullah_bin_sulaiman_al_rajhi,Abdullah bin Sulaiman Al Rajhi,1929-01-01,1929.0,NaN,NaN,93.0,Male,Saudi Arabia-Riyadh,Banking,Al Rajhi Banking & Investment;Al Rajhi Banking...,Saudi Arabia,sau,1929.0
4,abdulsamad_rabiu,Abdulsamad Rabiu,1960-08-04,1960.0,1960;1950,NaN,62.0,Male,Nigeria-Lagos,Construction Materials;Conglomerate;Multi Bran...,BUA;Bua,Nigeria,nga,1960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023,zuo_xiaoping,Zuo Xiaoping,1973-01-01,1973.0,NaN,NaN,49.0,Female,China-Guangdong-Foshan,Construction Materials;Industrial Products;Pla...,Lesso;L&S,China,chn,1973.0
5024,zuo_zongshen,Zuo Zongshen;Zuo Zongshen & family,1953-01-01,1953.0,1953,NaN,69.0,Male,China-Chongqing,"Auto Components;Motorcycles, Real Estate;Motor...",Zongshen,China,chn,1953.0
5025,zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,NaN,NaN,1953,NaN,NaN,NaN,NaN,"Motorcycles, Real estate",Zongshen,NaN,NaN,1953.0
5026,zygi_wilf,Zygi Wilf,1950-04-22,1950.0,1951,NaN,72.0,Male,United States-Millburn,Sports;Real Estate,Minnesota Vikings;Garden Homes,United States,usa,1950.0


In [112]:
hurun_ent[hurun_ent['person'] == 'adam_neuman']

,person,name,birth,birth_1,birth_2,birth_3,age,sex,permanent_country,industry,company,country,geo,birth_comb
16,adam_neuman,Adam Neuman,NaN,NaN,1980,NaN,NaN,NaN,NaN,Real Estate;Technology,WeWork,NaN,NaN,1980.0


In [113]:
def _f(x):
    if x in hurun_map.keys():
        return hurun_map[x]
    else:
        print(x)
        return np.nan

hurun_ent['gm_id'] = hurun_ent['person'].map(_f)

In [114]:
# map country name to geo. 
geo_syns = pd.read_csv('../../../ddf--open_numbers/ddf--synonyms--geo.csv')
geo_map = geo_syns.set_index('synonym')['geo'].to_dict()

geo_map['Eswatini (Swaziland)'] = 'swz'

In [115]:
def _f(x):
    if pd.isnull(x):
        return x
    else:
        if ';' in x:
            print(x)
            x_ = x.split(';')[0].strip()  ## first one is latest
            return geo_map[x_]
        else:
            return geo_map[x]

fbent_final['geo'] = fbent_final['country'].map(_f)

United States; Colombia
Israel; Kazakhstan
India; Thailand
Germany; United States
Czechia; Czech Republic
Lebanon; Saudi Arabia
Lebanon; Switzerland
Georgia; Russia
Canada; China
Hong Kong; China
Hong Kong; China
Czechia; Czech Republic
Brazil; United States
United States; Russia
Switzerland; Germany
Switzerland; Germany
Macau; Macao
Hong Kong; China
Israel; Russia
Switzerland; Australia
Singapore; Taiwan
United States; Canada
Cyprus; Norway
Czechia; Czech Republic
China; Hong Kong
Singapore; China
Monaco; Brazil
United States; China
Czechia; Czech Republic
United States; South Korea
Eswatini (Swaziland); Swaziland
China; Hong Kong
Ireland; India
Belgium; Kazakhstan
Czechia; Czech Republic
Czechia; Czech Republic
Czechia; Czech Republic
Czechia; Czech Republic
Austria; France
United Kingdom; Canada
Lebanon; Saudi Arabia
Hong Kong; China
Monaco; Italy
Cyprus; Greece
Cyprus; India
Canada; United Kingdom
Ukraine; Russia
United States; Russia
Germany; Russia
Hong Kong; China
Israel; Russia

In [116]:
fbent_final

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,geo
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,usa
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,usa
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,are
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,sau
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,idn
...,...,...,...,...,...,...,...,...,...,...,...,...
4227,harald_tschira,NaN,Germany,NaN,NaN,software; Software,Technology,NaN,NaN,NaN,Tschira,deu
4228,vinod_gupta,NaN,India,NaN,NaN,electrical equipment,Manufacturing,NaN,NaN,NaN,Gupta,ind
4229,anil_rai_gupta,NaN,India,NaN,NaN,electrical equipment,Manufacturing,NaN,NaN,NaN,Gupta,ind
4230,willi_liebherr,NaN,Switzerland,NaN,NaN,Construction; construction,Construction & Engineering,NaN,NaN,NaN,Liebherr,che


In [117]:
forbes_ent_1 = fbent_final.copy()
forbes_ent_1 = forbes_ent_1.dropna(subset=['person'])

In [118]:
hurun_ent_1 = hurun_ent.copy()
hurun_ent_1 = hurun_ent_1.dropna(subset=['gm_id'])

In [119]:
hurun_ent_1 = hurun_ent_1.drop(columns=['person']).set_index('gm_id')
hurun_ent_1.index.name = 'person'
hurun_ent_1 = hurun_ent_1.reset_index()

In [120]:
comb_ent = pd.concat([forbes_ent_1, hurun_ent_1], ignore_index=True)

In [121]:
comb_ent

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,geo,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb
0,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a_jerrold_perenchio,A. Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,NaN,NaN,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,are,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,NaN,NaN,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,sau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abdul_rasyid,Abdul Rasyid,Indonesia,NaN,NaN,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,idn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9255,zuo_xiaoping,Zuo Xiaoping,China,NaN,NaN,NaN,Construction Materials;Industrial Products;Pla...,NaN,49.0,NaN,NaN,chn,1973-01-01,1973.0,NaN,NaN,Female,China-Guangdong-Foshan,Lesso;L&S,1973.0
9256,zuo_zongshen,Zuo Zongshen;Zuo Zongshen & family,China,NaN,NaN,NaN,"Auto Components;Motorcycles, Real Estate;Motor...",NaN,69.0,NaN,NaN,chn,1953-01-01,1953.0,1953,NaN,Male,China-Chongqing,Zongshen,1953.0
9257,zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,NaN,NaN,NaN,NaN,"Motorcycles, Real estate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953,NaN,NaN,NaN,Zongshen,1953.0
9258,zygi_wilf,Zygi Wilf,United States,NaN,NaN,NaN,Sports;Real Estate,NaN,72.0,NaN,NaN,usa,1950-04-22,1950.0,1951,NaN,Male,United States-Millburn,Minnesota Vikings;Garden Homes,1950.0


In [122]:
gs = comb_ent.groupby('person')

In [123]:
gs.get_group('jeff_bezos')

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,geo,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb
1581,jeff_bezos,Jeff Bezos,United States,Washington,WA,Amazon; Amazon.com,Technology,jeff-bezos,58.0,M,Bezos,usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5990,jeff_bezos,Jeff Bezos,United States,NaN,NaN,NaN,E-Commerce;Online Retail,NaN,58.0,NaN,NaN,usa,1964-01-12,1964.0,1964,1964,Male,United States-Washington State-Seattle,Amazon,1964.0


In [124]:
gs.get_group('cai_kui')

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,geo,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb
430,cai_kui,Cai Kui,China,Sichuan,NaN,real estate,Real Estate,cai-kui,59.0,M,Cai,chn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4686,cai_kui,Cai Kui;Cai Kui & Family;Cai Kui & family,Hong Kong,NaN,NaN,NaN,Developer;Real Estate,NaN,61.0,NaN,NaN,hkg,1961-01-01,1961.0,NaN,NaN,Male,China-Hong Kong,Longfor,1961.0


In [125]:
hurun_ent[hurun_ent['gm_id'] == 'cai_kui']

,person,name,birth,birth_1,birth_2,birth_3,age,sex,permanent_country,industry,company,country,geo,birth_comb,gm_id
454,cai_kui,Cai Kui;Cai Kui & Family;Cai Kui & family,1961-01-01,1961.0,NaN,NaN,61.0,Male,China-Hong Kong,Developer;Real Estate,Longfor,Hong Kong,hkg,1961.0,cai_kui


In [126]:
# NOTE: because forbes is in front of hurun list,
# the combined result will have forbes propertties in front.

def group_func(df):
    res = dict()
    for c in df.columns:
        if c not in ['wealth', 'ranking', 'year']:
            values = [str(x).strip() for x in df[c].dropna().values]
            if values:
                if c == 'geo' and ('hkg' in values or 'twn' in values or 'mac' in values):
                    values.insert(0, 'chn')
                values = pd.Series(values).drop_duplicates(keep='first').values
                res[c] = ";".join(values)
            else:
                res[c] = ''
    return pd.Series(res)

In [127]:
group_func(gs.get_group('cai_kui'))

person                                                         cai_kui
name                 Cai Kui;Cai Kui;Cai Kui & Family;Cai Kui & family
country                                                China;Hong Kong
state                                                          Sichuan
headquarters                                                          
source                                                     real estate
industry                             Real Estate;Developer;Real Estate
image_uri                                                      cai-kui
age                                                          59.0;61.0
gender                                                               M
last_name                                                          Cai
geo                                                            chn;hkg
birth                                                       1961-01-01
birth_1                                                         1961.0
birth_

In [128]:
# TODO: for now keep all columns.. need to cleanup

In [129]:
res_ent = gs.apply(group_func)

In [130]:
res_ent

,person,name,country,state,headquarters,source,industry,image_uri,age,gender,last_name,geo,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb
person,,,,,,,,,,,,,,,,,,,,
a_jayson_adair,a_jayson_adair,A. Jayson Adair,United States,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,usa,,,,,,,,
a_jerrold_perenchio,a_jerrold_perenchio,A. Jerrold Perenchio;Jerrold Perenchio,United States,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media;Entertainment,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,usa,,,1931,,,,Chartwell Leisure,1931.0
abdul_aziz_al_ghurair,abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,,,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,are,,,,,,,,
abdul_majeed_alhokair,abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,,,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,sau,,,,,,,,
abdul_rasyid,abdul_rasyid,Abdul Rasyid,Indonesia,,,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,idn,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuo_zongshen,zuo_zongshen,Zuo Zongshen;Zuo Zongshen & family,China,,,,"Auto Components;Motorcycles, Real Estate;Motor...",,69.0,,,chn,1953-01-01,1953.0,1953,,Male,China-Chongqing,Zongshen,1953.0
zuo_zongshen_and_yuan_dexiu,zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,,,,,"Motorcycles, Real estate",,,,,,,,1953,,,,Zongshen,1953.0
zuowen_song,zuowen_song,Zuowen Song;Song Zuowen;Song Zuowen & Family;S...,China,,,"aluminum, diversified",Diversified;Non-ferrous metals processing;Meta...,zuowen-song,75.0,M,Song,chn,1947-03-01,1947.0,1947,,Male,China-Shandong-Yantai,Nanshan,1947.0


In [131]:
# res_ent = res_ent.drop(columns=['gm_id'])

In [132]:
res_ent.loc['cai_kui']

person                                                         cai_kui
name                 Cai Kui;Cai Kui;Cai Kui & Family;Cai Kui & family
country                                                China;Hong Kong
state                                                          Sichuan
headquarters                                                          
source                                                     real estate
industry                             Real Estate;Developer;Real Estate
image_uri                                                      cai-kui
age                                                          59.0;61.0
gender                                                               M
last_name                                                          Cai
geo                                                            chn;hkg
birth                                                       1961-01-01
birth_1                                                         1961.0
birth_

In [133]:
res_ent = res_ent.drop(columns=['person'])

In [134]:
# res_ent.index.name = 'person'

In [135]:
res_ent['countries'] = res_ent['geo']
res_ent['names'] = res_ent['name']

In [136]:
def _f(x):
    if isinstance(x, str):
        if ';' in x:
            return x.split(';')[0].strip()
    return x

res_ent['name'] = res_ent['names'].map(_f)

In [137]:
res_ent = res_ent.drop(columns=['geo'])

In [138]:
res_ent = res_ent.drop(columns=['country'])

In [139]:
res_ent.columns

Index(['name', 'state', 'headquarters', 'source', 'industry', 'image_uri',
       'age', 'gender', 'last_name', 'birth', 'birth_1', 'birth_2', 'birth_3',
       'sex', 'permanent_country', 'company', 'birth_comb', 'countries',
       'names'],
      dtype='object')

In [140]:
# drop those entities without country
res_ent = res_ent[~pd.isnull(res_ent['countries'])]

In [141]:
res_ent

,name,state,headquarters,source,industry,image_uri,age,gender,last_name,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb,countries,names
person,,,,,,,,,,,,,,,,,,,
a_jayson_adair,A. Jayson Adair,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,,,,,,,,,usa,A. Jayson Adair
a_jerrold_perenchio,A. Jerrold Perenchio,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media;Entertainment,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,,,1931,,,,Chartwell Leisure,1931.0,usa,A. Jerrold Perenchio;Jerrold Perenchio
abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,,,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,,,,,,,,,are,Abdul Aziz Al Ghurair
abdul_majeed_alhokair,Abdul Majeed Alhokair,,,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,,,,,,,,,sau,Abdul Majeed Alhokair
abdul_rasyid,Abdul Rasyid,,,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,,,,,,,,,idn,Abdul Rasyid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuo_zongshen,Zuo Zongshen,,,,"Auto Components;Motorcycles, Real Estate;Motor...",,69.0,,,1953-01-01,1953.0,1953,,Male,China-Chongqing,Zongshen,1953.0,chn,Zuo Zongshen;Zuo Zongshen & family
zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,,,,"Motorcycles, Real estate",,,,,,,1953,,,,Zongshen,1953.0,,Zuo Zongshen & Yuan Dexiu
zuowen_song,Zuowen Song,,,"aluminum, diversified",Diversified;Non-ferrous metals processing;Meta...,zuowen-song,75.0,M,Song,1947-03-01,1947.0,1947,,Male,China-Shandong-Yantai,Nanshan,1947.0,chn,Zuowen Song;Song Zuowen;Song Zuowen & Family;S...


In [142]:
res_ent.to_csv('../../ddf--entities--person.csv')

In [143]:
res_ent

,name,state,headquarters,source,industry,image_uri,age,gender,last_name,birth,birth_1,birth_2,birth_3,sex,permanent_country,company,birth_comb,countries,names
person,,,,,,,,,,,,,,,,,,,
a_jayson_adair,A. Jayson Adair,Texas,TX,damaged cars,Automotive,no-pic,52.0,M,Adair,,,,,,,,,usa,A. Jayson Adair
a_jerrold_perenchio,A. Jerrold Perenchio,California,CA,"television, Univision; television; Univision",Media & Entertainment; Media;Entertainment,a-jerrold-perenchio; no-pic,91.0,M,Perenchio,,,1931,,,,Chartwell Leisure,1931.0,usa,A. Jerrold Perenchio;Jerrold Perenchio
abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,,,banking,Finance,abdul-aziz-al-ghurair; no-pic,67.0,M,Al Ghurair,,,,,,,,,are,Abdul Aziz Al Ghurair
abdul_majeed_alhokair,Abdul Majeed Alhokair,,,"retail, real estate",Fashion & Retail,no-pic,52.0,M,Alhokair,,,,,,,,,sau,Abdul Majeed Alhokair
abdul_rasyid,Abdul Rasyid,,,"timber, palm oil",Manufacturing,abdul-rasyid,64.0,M,Abdul,,,,,,,,,idn,Abdul Rasyid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuo_zongshen,Zuo Zongshen,,,,"Auto Components;Motorcycles, Real Estate;Motor...",,69.0,,,1953-01-01,1953.0,1953,,Male,China-Chongqing,Zongshen,1953.0,chn,Zuo Zongshen;Zuo Zongshen & family
zuo_zongshen_and_yuan_dexiu,Zuo Zongshen & Yuan Dexiu,,,,"Motorcycles, Real estate",,,,,,,1953,,,,Zongshen,1953.0,,Zuo Zongshen & Yuan Dexiu
zuowen_song,Zuowen Song,,,"aluminum, diversified",Diversified;Non-ferrous metals processing;Meta...,zuowen-song,75.0,M,Song,1947-03-01,1947.0,1947,,Male,China-Shandong-Yantai,Nanshan,1947.0,chn,Zuowen Song;Song Zuowen;Song Zuowen & Family;S...


In [144]:
res_ent.countries.unique()

array(['usa', 'are', 'sau', 'idn', 'nga', 'ind', 'bra', 'chn;hkg', '',
       'isr;usa', 'rou', 'dnk', 'tur', 'chn', 'gbr', 'rus', 'tha', 'jpn',
       'can', 'fra', 'gbr;che', 'che', 'aus', 'deu', 'bel', 'esp', 'mex',
       'per', 'ita', 'arg', 'isr', 'kaz', 'nor', 'prt', 'rus;gbr', 'swe',
       'nld', 'rus;che', 'zaf', 'ind;tha', 'chl', 'mys', 'mar', 'bra;gbr',
       'chn;twn', 'nor;usa', 'deu;usa', 'cze', 'ury', 'phl', 'ukr',
       'ind;gbr', 'fra;che', 'fin', 'can;usa', 'mco', 'grc;che', 'sgp',
       'deu;che', 'ita;che', 'lbn;sau', 'lbn;che', 'kor', 'kwt;usa',
       'col', 'che;fra', 'swe;che', 'geo', 'npl', 'usa;aus', 'usa;kor',
       'ind;are', 'aus;bhs', 'vnm', 'gbr;usa', 'fra;gbr', 'can;chn',
       'nld;gbr', 'mys;khm', 'chn;twn;usa', 'deu;gbr', 'lie', 'nzl;are',
       'cyp;mco', 'rou;usa', 'esp;che', 'pol', 'chn;mac', 'isl;dnk',
       'col;bra', 'usa;chn', 'irl', 'aut', 'rus;sgp', 'rus;mco',
       'idn;usa', 'bra;sgp', 'bel;mco', 'isr;mco', 'lbn;fra', 'qat',
      

In [145]:
res_ent[res_ent.countries.str.contains(';')]['countries'].to_dict()

{'adam_kwok': 'chn;hkg',
 'adam_kwok_kai_fai': 'chn;hkg',
 'adam_neumann': 'isr;usa',
 'alan_howard': 'gbr;che',
 'albert_yeung': 'chn;hkg',
 'alexei_kuzmichev': 'rus;gbr',
 'alisher_usmanov': 'rus;che',
 'allan_wong': 'chn;hkg',
 'aloke_lohia': 'ind;tha',
 'andre_esteves': 'bra;gbr',
 'andre_koo_sr': 'chn;twn',
 'andreas_halvorsen': 'nor;usa',
 'andreas_von_bechtolsheim': 'deu;usa',
 'andrew_currie': 'gbr;che',
 'angela_leong': 'chn;hkg',
 'anil_agarwal': 'ind;gbr',
 'anne_beaufour': 'fra;che',
 'antonio_luiz_seabra': 'bra;gbr',
 'apoorva_mehta': 'can;usa',
 'archie_hwang': 'chn;twn',
 'aristotelis_mistakidis': 'grc;che',
 'august_von_finck': 'deu;che',
 'augusto_giorgio_perfetti': 'ita;che',
 'ayman_hariri': 'lbn;sau',
 'bahaa_hariri': 'lbn;che',
 'barry_lam': 'chn;twn',
 'bassam_alghanim': 'kwt;usa',
 'benjamin_de_rothschild': 'che;fra',
 'bertil_hult': 'swe;che',
 'billy_kan': 'chn;hkg',
 'blair_parry_okeden': 'usa;aus',
 'bom_kim': 'usa;kor',
 'br_shetty': 'ind;are',
 'brett_blund